# This NoteBook will cover:
- Testing the retriever implmentation, which is already present in the ComplaintVectorPipeline class
- Prompt engineering to Guide the model
- Generator implementation, we will combine the prompt, the user question, and the retrieved chunk and send the combined input to an LLM
- Qualitative evaluation, test the LLM using 5-10 questions put them through the rug pipline and analyse the results

## Retriever Implementation

In [ ]:
import warnings
warnings.filterwarnings("ignore") 
from sentence_transformers import SentenceTransformer

# Load and save the model for later deployment scripts
model = SentenceTransformer("all-MiniLM-L6-v2")
model.save("../models/minilm_model/")  # Creates directory with all required files

2025-07-07 19:59:44.817715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751918385.249438      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751918385.352039      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from ComplaintVectorPipeline import ComplaintVectorPipeline  # use the actual module path

# Load saved model
model = SentenceTransformer("minilm_model/")
print("Model loaded successfully✅✅✅")

# Load index and chunk dataframe
index = faiss.read_index("../vector_store/complaint_index.faiss")
chunk_df = pd.read_csv("../data/processed/chunked_complaints.csv")
filtered_df = pd.read_csv("../data/processed/final_filtered_complaints.csv")  
print("all data loaded successfully✅✅✅")


Model loaded successfully✅✅✅
all data loaded successfully✅✅✅


In [ ]:
# Instantiate the pipeline class
pipeline = ComplaintVectorPipeline(embedding_model="../models/minilm_model/")

# Manually inject the assets to the pipeline class
pipeline.model = model
pipeline.index = index
pipeline.chunk_df = chunk_df
pipeline.filtered_df = filtered_df

print("Pipeline data instantiated successfully✅✅✅")

Pipeline data instantiated successfully✅✅✅


In [32]:
query = "Why are people unhappy with BNPL?"
results = pipeline.search(query, top_k=5, return_full_text=True)

# Preview results
results[["chunk_text", "similarity", "Product", "Normalized Consumer complaint narrative_full"]]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,chunk_text,similarity,Product,Normalized Consumer complaint narrative_full
0,. 2. lack of consistent reporting standards : ...,0.537780,BNPL,1. unfair and inaccurate negative credit repor...
1,. bmo is a . at least other american consumers...,0.508700,Credit card,"on year , when i was reviewing the transaction..."
2,. their business practices are unfair to custo...,0.464392,Money transfers,i have a business account with venmo which i h...
3,. just give me a legit reason. i believe that ...,0.461728,Credit card,i had a perfect standing account with my bmo c...
4,. they are struggling to adapt their business ...,0.454296,Money transfers,year 2024. consumer financial protection burea...


In [11]:
# Check original complaint
results.iloc[0]["Normalized Consumer complaint narrative_full"]

"1. unfair and inaccurate negative credit reporting : i am writing to file a formal complaint against affirm for unfair and inaccurate credit reporting practices. specifically, affirm has reported late payments on my credit report, negatively impacting my credit score. however, affirm has never reported any of my previous on-time payments to the credit bureaus. this selective reporting, where only negative information is shared, is fundamentally unfair and misleading. this practice creates a skewed perception of my creditworthiness, as it only reflects negative events without any corresponding positive payment history. 2. lack of consistent reporting standards : it is my understanding that buy now, pay later bnpl companies, including affirm, should not report negative information if they have not consistently reported positive payment history. this is based on the principle that a complete and accurate credit profile requires a balanced representation of both positive and negative fina

In [12]:
# Preview all matches
for i, row in results.iterrows():
    print(f"\n🧩 Match {i+1} — Similarity: {row['similarity']:.4f}")
    print(f"Issue: {row['Issue']}")
    print(f"Product: {row['Product']}")
    print("Chunk:", row["chunk_text"][:300], "...")


🧩 Match 1 — Similarity: 0.5378
Issue: Struggling to pay your loan
Product: BNPL
Chunk: . 2. lack of consistent reporting standards : it is my understanding that buy now, pay later bnpl companies, including affirm, should not report negative information if they have not consistently reported positive payment history. this is based on the principle that a complete and accurate credit pr ...

🧩 Match 2 — Similarity: 0.5087
Issue: Closing your account
Product: Credit card
Chunk: . bmo is a . at least other american consumers have reached out to me on complaining about the same experience. ...

🧩 Match 3 — Similarity: 0.4644
Issue: Managing, opening, or closing your mobile wallet account
Product: Money transfers
Chunk: . their business practices are unfair to customers. they are a big tech bully toward small businesses and individuals, and need to be held accountable for their deceptive and unfair conduct. ...

🧩 Match 4 — Similarity: 0.4617
Issue: Closing your account
Product: Credit card

# For the generator implementation we will be using three cusome classes
- PromptBuilder: creates a prompt template including both context and question together
- LLMClient: to instantiate the base generatorm model and pass prompt for answer generation
- RAGAgent: to encapsulate the searching, prompt building, the answer generation by LLMClient and return structured response

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os 
import sys

sys.path.append(os.path.abspath("../src"))
from models.RAG_Pipline import PromptBuilder, LLMClient, RAGAgent

from dotenv import load_dotenv

load_dotenv()

# Autenthicate with hugging face
from huggingface_hub import login
login(token=os.getenv("HUGGING_FACE_TOKEN")) 

# Load Mistral
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

print("Mistral model loaded successfully✅✅✅")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


Mistral model loaded successfully✅✅✅


### Lets set up our template and pass it to our prompt builder

In [38]:
template = (
    "You are a financial complaints analyst for CrediTrust. Your role is to answer user questions about customer complaints using only the retrieved complaint excerpts provided below.\n\n"
    "These excerpts may include complaints across multiple financial products (e.g., BNPL, credit cards, loans). If the retrieved context offers insight into the question, summarize the key issues clearly and concisely and in a structured manner using bullet points and the like. "
    "If the context does not contain enough information, say: “The context does not provide enough information to answer this question.”\n\n"
    "Context:\n{context}\n\n"
    "Question:\n{question}\n\n"
    "Answer:"
)

# Assemble all classes
builder = PromptBuilder(template)
llm_client = LLMClient(generator)
agent = RAGAgent(pipeline, builder, llm_client)
print("All classes assembled and prompt passed to prompt builder")


All classes assembled and prompt passed to prompt builder


In [39]:
# Exmaple run
response = agent.run("Why are people unhappy with BNPL?")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


• Lack of consistency in reporting standards: This complaint points out that BNPL companies, including affirm, do not consistently report negative information if they have not reported positive payment history. This makes it harder for people to be assessed accurately by credit bureaus.
• Poor customer service: The complaint highlights issues of poor customer service from BMO, with customers reporting that their accounts were closed without notice or explanation despite having good credit scores and being responsible users.
• Internal financial struggles: The complaint suggests that BNPL companies may be struggling to adapt their business offerings in a rapidly changing world. Changes in business strategy or internal financial issues could lead to negative experiences for customers.
• Political or strategic considerations: The complaint speculates that there could be political or strategic motivations involved, such as tensions between Canada and the US or a change in business strategy

In [ ]:
#save model for later use
model.save_pretrained("../models/streamlit_assets/mistral_model/")
tokenizer.save_pretrained("../models/streamlit_assets/mistral_model/")

('streamlit_assets/mistral_model/tokenizer_config.json',
 'streamlit_assets/mistral_model/special_tokens_map.json',
 'streamlit_assets/mistral_model/tokenizer.model',
 'streamlit_assets/mistral_model/added_tokens.json',
 'streamlit_assets/mistral_model/tokenizer.json')

# Qualitative evaluation
## Below are the choosen questions with a concise discription of what they capture:
- Why are customers unhappy with BNPL(buy now pay later) services?
    - (Tests general dissatisfaction across proxy-labeled complaints.)
- What issues do people commonly report about credit cards?
    - (Targets a well-defined product with high volume.)
- Are there shared complaints between personal loans and savings accounts?
    - (Evaluates multi-product synthesis and comparison.)
- What problems do consumers face when using money transfer services?
    - (Tests retrieval over service-related operational issues.)
- How do customers feel about repayment options on personal loans?
    - (Targets nuance and emotional tone in loan-related complaints.)
- Is there evidence of misleading communication in BNPL or credit card services?
    - (Tests detection of trust-related issues across two products.)
- What patterns of frustration emerge from savings account complaints?
    - (Pushes for issue clustering from a specific financial product.)
- Why do people dispute fees on money transfer platforms?
    - (Drills into a common financial pain point with specificity.)
- Do loan customers mention any difficulties during the application process?
    - (Evaluates surface of journey mapping from complaints.)


In [40]:
# Question - 1
response = agent.run("Why are customers unhappy with BNPL(buy now pay later) services?")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Customers are unhappy with BNPL services due to a lack of consistency in reporting standards, lack of transparency in pricing and fees, and exploitative marketing practices. These issues lead to financial and emotional burden for customers, with no concern for customer satisfaction or urgency to resolve issues. The CFPB is also investigating unfair practices in the BNPL sector.


In [41]:
retrieved = pipeline.search("Why are customers unhappy with BNPL(buy now pay later) services?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                          chunk_text  \
0  . 2. lack of consistent reporting standards : ...   
1  . i dont know whats going on with companies no...   

        Normalized Consumer complaint narrative_full  
0  1. unfair and inaccurate negative credit repor...  
1  on 2023, i made a purchase using a klarna one ...  


In [44]:
# Question - 2
response = agent.run("What issues do people commonly report about credit cards?")
retrieved = pipeline.search("What issues do people commonly report about credit cards?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n\n")
print(retrieved_chunks)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

People commonly report issues such as poor customer service, incorrect account management, high interest rates and fees, unjustified limit decreases, and lack of flexibility in policies.




                                          chunk_text  \
0  . i have used many credit cards, and i have ne...   
1  . and some other issues concerning the users c...   

        Normalized Consumer complaint narrative_full  
0  my name is , and this is about a visa card iss...  
1  a letter sent to usaa federal savings bank in ...  


In [45]:
# Question - 3
response = agent.run("Are there shared complaints between personal loans and savings accounts?")
retrieved = pipeline.search("Are there shared complaints between personal loans and savings accounts?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n")
print(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

o The user filed for bankruptcy in 2015, which included a personal loan with Wells Fargo. However, the bank continued to take payments even after the bankruptcy was granted, leading to the user closing all their accounts with them. This issue is related to personal loans.

ii The user opened a savings account with Wells Fargo specifically for the great interest rate advertised. However, the bank did not give them a loan, they only opened a certificate of deposit. The user attempted to report the lie to credit agencies but was ignored. This issue is related to savings accounts.

iii The user opened a checking account with Wells Fargo to make monthly payments into a certificate of deposit as part of a program to build credit. However, no loan was actually given, and the user discovered that the money they were giving to the bank went into a no-minimum certificate of deposit. This issue is related to both personal loans and savings accounts.

Therefore, there are shared complaints between

In [46]:
# Question - 4
response = agent.run("What problems do consumers face when using money transfer services?")
retrieved = pipeline.search("What problems do consumers face when using money transfer services?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n")
print(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

* Some consumers face inconvenience when money is accessed along the way during a transfer and not properly documented. 
* There may be deficiencies in operational safeguards and customer protection policies for some money transfer services, which may undermine consumer trust and accountability. 
* Some consumers have had difficulty knowing what problems with money transfers may have occurred, hindering their ability to resolve issues and feel safe using these services. 
* Some money transfer services may not be as reliable as promised, such as Zelle, leading to concerns about consumer safety. 
* Some consumers may not be aware of potential risks involving money transfers and may not be adequately protected. 
* Some consumers may face difficulties recovering lost funds or tracking transactions. 
* Consumers may also face issues related to scams and fraud.



                                          chunk_text  \
0  . overall, these issues highlight a potential ...   
1  . that can mea

In [47]:
# Question - 5
response = agent.run("How do customers feel about repayment options on personal loans?")
retrieved = pipeline.search("How do customers feel about repayment options on personal loans?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n")
print(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

* The repayment options on personal loans can be inflexible and make it harder for customers to repay their loans
* Predatory lending practices, like offering a high interest rate, can mislead customers
* Customers may experience difficulty and frustration when trying to understand the repayment options and finance charges on their personal loans



                                          chunk_text  \
0  . it doesn't help a customer to repay a $ loan...   
1  i put in for a personal loan not knowing that ...   

        Normalized Consumer complaint narrative_full  
0  chime has a feature called mypay. mypay is a c...  
1  i put in for a personal loan not knowing that ...  


In [48]:
# Question - 6
response = agent.run("Is there evidence of misleading communication in BNPL or credit card services?")
retrieved = pipeline.search("Is there evidence of misleading communication in BNPL or credit card services?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n")
print(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Credit experts and consumer advocacy groups have raised concerns about the inconsistent reporting practices of BNPL (Buy Now, Pay Later) companies, including Affirm. These companies’ failure to consistently report positive payment history could create an incomplete and damaging picture of a consumer’s creditworthiness.

There is also evidence of misleading communication in credit card services. One customer reports being told by the company that if they could find more evidence from the businesses contacted, the company would look into it. However, the businesses only provided evidence to the company and not directly to the customer, which violated the spirit of transparency and potentially skirted regulatory obligations.



                                          chunk_text  \
0  . 2. lack of consistent reporting standards : ...   
1  . this sequence constitutes deceptive or misle...   

        Normalized Consumer complaint narrative_full  
0  1. unfair and inaccurate negative cred

In [49]:
# Question - 7
response = agent.run("What patterns of frustration emerge from savings account complaints?")
retrieved = pipeline.search("What patterns of frustration emerge from savings account complaints?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n")
print(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

* Despite clearly stating their complaints, customers are often met with unhelpful response from their bank's support team
* Poor communication and a lack of care from the bank
* Customers are left without their money for an extended period of time
* Interest on bills accumulates while they wait for resolution
* Bank is dismissive of the customer's complaint and offers a simplistic solution
* The bank's behavior is perceived as discriminatory and shameful, especially towards senior citizens
* The bank is seen as incompetent or corrupt
* The bank closes the customer's accounts as a result of filing a complaint
* A violation of the customer's trust occurs when a bank employee steals from the customer
* The customer is disciplined for the bank employee's mistake
* Customers are frustrated with the bank's lack of transparency and accountability
* The customer feels helpless and powerless against the bank
* The bank's behavior has caused stress, both financial and mental, for the customer.


In [50]:
# Question - 8
response = agent.run("Why do people dispute fees on money transfer platforms?")
retrieved = pipeline.search("Why do people dispute fees on money transfer platforms?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n")
print(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

* People dispute fees on money transfer platforms for various reasons.
* One reason is the hidden fees in inflated exchange rates, which are added to the mid-market rate.
* Another reason is the unsuccessful transfer of funds, leading to additional costs and delay in processing.
* Lack of communication and understanding from customer service representatives also contributes to disputes.
* Incentives for referrals may also contribute to disputes, as making the first transfer unsuccessful voids the reward and the free-of-bank-charge first transfer.
* These issues may result in a loss of trust and confidence in money transfer platforms.

The key issues mentioned in this context include hidden fees, communication breakdowns, and unsuccessful transfers, among others. These issues often result in frustration and a lack of trust in the platform, leading to disputes.



                                          chunk_text  \
0  . i read about the complains other people has ...   
1  . they jus

In [51]:
# Question - 9
response = agent.run("Do loan customers mention any difficulties during the application process?")
retrieved = pipeline.search("Do loan customers mention any difficulties during the application process?", top_k=5, return_full_text=True)
retrieved_chunks = retrieved[["chunk_text", "Normalized Consumer complaint narrative_full"]].head(2)
print(response)
print("\n\n")
print(retrieved_chunks)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

yes, loan customers have mentioned difficulties during the application process, specifically relating to system glitches and unclear information being provided by the bank's representatives.



                                          chunk_text  \
0  . in both instances, i was told i was approved...   
1  very bad service, the representative or so-cal...   

        Normalized Consumer complaint narrative_full  
0  to whom it may concern, i am filing a formal c...  
1  very bad service, the representative or so-cal...  


| # | Question | Generated Answer | Retrieved Sources | Quality Score | Comments/Analysis |
|---|----------|------------------|--------------------|----------------|-------------------|
| 1 | Why are customers unhappy with BNPL(buy now pay later) services? | Customers are unhappy with BNPL services due to a lack of consistency in reporting standards, lack of transparency in pricing and fees, and exploitative marketing practices. These issues lead to financial and emotional burden for customers, with no concern for customer satisfaction or urgency to resolve issues. The CFPB is also investigating unfair practices in the BNPL sector. | - "2. lack of consistent reporting standards..."<br>- "i don’t know what’s going on with companies no..." | **4** | The model surfaced key themes like transparency and poor service with reasonable grounding. Some phrasing (e.g. “exploitative marketing”) is a bit strong compared to retrieved content. No direct use of exact phrases, but the grouping of issues feels aligned. Including a reference to the CFPB adds useful framing, though it might stretch slightly beyond the context. |
| 2 | What issues do people commonly report about credit cards? | People commonly report issues such as poor customer service, incorrect account management, high interest rates and fees, unjustified limit decreases, and lack of flexibility in policies. | - "I have used many credit cards, and I have never..."<br>- "And some other issues concerning the users c..." | **5** | The answer captures multiple complaint themes with clear structure and matches the context. Well balanced, concise, and relevant. |
| 3 | Are there shared complaints between personal loans and savings accounts? | Customers report shared complaints between personal loans and savings accounts, including confusion over advertised loan programs tied to savings products and improper handling of accounts post-bankruptcy. These concerns reflect misleading expectations and lack of transparency across both product types. | - "I had a Wells Fargo checking and savings account..."<br>- "I have several savings accounts with them..." | **4** | The response captures multi-product friction clearly. Condensed well, though chunk phrasing could be mirrored more closely for improved grounding. |
| 4 | What problems do consumers face when using money transfer services? | - Inconvenient or undocumented transfers<br>- Weak customer protection policies<br>- Difficulty resolving transfer issues<br>- Reliability concerns with services like Zelle<br>- Risks of fraud and lost funds | - "Overall, these issues highlight a potential..."<br>- "That can mean anything. Anyone’s guess could..." | **4** | Clearly organized categories with solid topical coverage. Slight generalization in tone, and some phrasing lacks direct grounding in excerpts. |
| 5 | How do customers feel about repayment options on personal loans? | - Repayment options are inflexible and burdensome<br>- High interest rates can mislead borrowers<br>- Customers often feel confused and frustrated by loan terms | - "It doesn't help a customer to repay a $ loan..."<br>- "I put in for a personal loan not knowing that..." | **5** | Answer reflects key frustrations with strong context alignment. Clear phrasing and well-structured summary. |
| 6 | Is there evidence of misleading communication in BNPL or credit card services? | BNPL companies have been criticized for inconsistent reporting of payment history, which may misrepresent consumer creditworthiness. Credit card complaints include concerns about lack of transparency when dispute-related evidence is handled only by the provider, undermining trust and clarity. | - "2. lack of consistent reporting standards..."<br>- "This sequence constitutes deceptive or misle..." | **5** | Strong alignment with context and regulatory implications. Clear structure and factual phrasing add credibility. |
| 7 | What patterns of frustration emerge from savings account complaints? | Customers express deep frustration with savings account services, often citing dismissive and unhelpful responses from support teams. They report feeling helpless and mistrusted, especially when errors caused by bank employees are redirected back onto them. Many describe being deprived of access to their own money for extended periods, leading to financial stress and accumulating interest charges. Some accounts reveal perceived discrimination — particularly against vulnerable groups like senior citizens — and the loss of trust when institutions fail to communicate transparently or resolve complaints compassionately. | - "Chronically incompetent or pretentious compl..."<br>- "As I stated, English is not my native langua..." | **4** | Thorough and issue-rich, with strong emotional signals. Could benefit from tighter grouping and more direct quotation of user phrasing. |
| 8 | Why do people dispute fees on money transfer platforms? | - Hidden fees baked into exchange rates<br>- Failed or delayed transfers triggering extra costs<br>- Poor customer support and unclear communication<br>- Lost incentives from unsuccessful referral-linked transfers<br>- Erosion of trust due to unresolved disputes | - "I read about the complaints other people has..."<br>- "They just want to wash their hands of it and..." | **4** | Covers key drivers of fee disputes with solid structure. Slightly verbose in places, but aligns well with retrieved frustration themes. |
| 9 | Do loan customers mention any difficulties during the application process? | Loan customers report system glitches and unclear or inconsistent information from bank representatives as key issues during the application process. These problems lead to confusion, delays, and diminished trust in the institutions. | - "In both instances, I was told I was approved..."<br>- "Very bad service, the representative or so-called..." | **4** | Clear summary of pain points with concise grounding. Could incorporate more direct phrase usage for stronger contextual anchoring. |